In [2]:
import psycopg2
import json
import geopandas as gpd
import pandas as pd

# ✅ Database Connection Using psycopg2
POSTGIS_CONN = {
    "dbname": "crowdcounting",
    "user": "admin",
    "password": "admin",
    "host": "40.84.231.179",
    "port": "5434",
}

In [3]:
roi = gpd.read_file('playgroundROI.gpkg')
roi.explore()

In [5]:
import geopandas as gpd
import pandas as pd
import psycopg2
from shapely.ops import unary_union

# 1) Carga ROI y asegúrate de EPSG:4326
roi = gpd.read_file('playgroundROI.gpkg')
if roi.crs is None:
    raise ValueError("El GeoPackage no tiene CRS. Asigna uno correcto o reproyecta.")
if roi.crs.to_epsg() != 4326:
    roi = roi.to_crs(epsg=4326)

# Un solo polígono para la consulta (une si hay varios)
roi_geom = unary_union(roi.geometry)
roi_wkt = roi_geom.wkt  # lo enviaremos como parámetro WKT

# 2) Consulta con ST_Intersects: devolvemos filas top N por cámara y el total global
# - Añadimos total_matches usando COUNT(*) OVER() para que no falle el acceso en Python
# - Eliminamos el LIMIT 10 que truncaba resultados inesperadamente; aplicamos LIMIT 1000 al final
SQL_QUERY = """
WITH roi as (
  SELECT ST_GeomFromText(%s, 4326) AS geom
),
filtered as (
  SELECT p.*
  FROM person_observed p
  JOIN roi r
    ON ST_Intersects(p.geom, r.geom)
)
SELECT
  *,
  COUNT(*) OVER() AS total_matches
FROM filtered
ORDER BY timestamp
"""

try:
    conn = psycopg2.connect(**POSTGIS_CONN)
    df = pd.read_sql(SQL_QUERY, conn, params=[roi_wkt])
    conn.close()
except Exception as e:
    print(f"Error connecting to database: {e}")
    df = pd.DataFrame()

# 3) Mostrar resultados y total
if not df.empty:
    # total_matches ahora está en la consulta; puede obtenerse desde la primera fila
    total = int(df['total_matches'].iloc[0])
    print(f"Total dentro del ROI: {total}")
    print(f"Mostrando {len(df)} filas (máximo 1000).")
else:
    print("No hubo coincidencias dentro del ROI o la consulta falló.")


/tmp/ipykernel_105220/3362919127.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(SQL_QUERY, conn, params=[roi_wkt])


Total dentro del ROI: 1592490
Mostrando 1592490 filas (máximo 1000).


In [6]:
df.head(20)

,id,id_person,lat,long,timestamp,geom,camera_name,coordinate_x,coordinate_y,tracklet_id,total_matches
0,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653059,-100.286003,2024-09-25 12:22:23.666686+00:00,0101000020E6100000BCFFFFDF4D1259C0AAFFFFDF2EA7...,None,NaN,NaN,None,1592490
1,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653059,-100.286003,2024-09-25 12:22:23.708353+00:00,0101000020E6100000BCFFFFDF4D1259C0AAFFFFDF2EA7...,None,NaN,NaN,None,1592490
2,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653057,-100.286003,2024-09-25 12:22:23.750021+00:00,0101000020E6100000BCFFFFDF4D1259C043FFFFBF2EA7...,None,NaN,NaN,None,1592490
3,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653057,-100.286003,2024-09-25 12:22:23.791689+00:00,0101000020E6100000BCFFFFDF4D1259C043FFFFBF2EA7...,None,NaN,NaN,None,1592490
4,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653055,-100.286003,2024-09-25 12:22:23.833357+00:00,0101000020E6100000BCFFFFDF4D1259C0F5FFFF9F2EA7...,None,NaN,NaN,None,1592490
5,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653053,-100.286003,2024-09-25 12:22:23.875025+00:00,0101000020E6100000BCFFFFDF4D1259C08EFFFF7F2EA7...,None,NaN,NaN,None,1592490
6,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653051,-100.286011,2024-09-25 12:22:23.916692+00:00,0101000020E6100000DDFFFFFF4D1259C026FFFF5F2EA7...,None,NaN,NaN,None,1592490
7,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653049,-100.286011,2024-09-25 12:22:23.958360+00:00,0101000020E6100000DDFFFFFF4D1259C0D8FFFF3F2EA7...,None,NaN,NaN,None,1592490
8,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653048,-100.286011,2024-09-25 12:22:24.000028+00:00,0101000020E6100000DDFFFFFF4D1259C071FFFF1F2EA7...,None,NaN,NaN,None,1592490
9,columpioscam3-2024-09-25_062223.mp4:b17e1ad8-0...,b17e1ad8-0c2c-44e4-ba72-54c2dd3881a4,25.653048,-100.286011,2024-09-25 12:22:24.041696+00:00,0101000020E6100000DDFFFFFF4D1259C071FFFF1F2EA7...,None,NaN,NaN,None,1592490


In [7]:
import os
from datetime import datetime
from azure.storage.blob import BlobServiceClient
from tqdm import tqdm

account_url = 'https://cienciaciudades2024.blob.core.windows.net'
container = 'crowdcounting'
sas_token = 'sp=racwdl&st=2025-02-05T16:48:09Z&se=2026-04-02T00:48:09Z&spr=https&sv=2022-11-02&sr=c&sig=pXOsajenPg9lvILk6OozoyE%2BUz%2FDXivaBWwIioLyRVo%3D'
blob_service_client = BlobServiceClient(account_url, credential=sas_token)

# Build list of azure paths from the DataFrame (preserve order, unique values)
azure_paths = []
if 'video_path' in df.columns and df['video_path'].notna().any():
    seen = set()
    for v in df['video_path'].dropna().astype(str).tolist():
        if v and v not in seen:
            seen.add(v)
            azure_paths.append(v)
else:
    seen = set()
    for s in df['id'].dropna().astype(str).tolist():
        p = s.split(':')[0] if ':' in s else s
        if p and p not in seen:
            seen.add(p)
            azure_paths.append(p)

print(f"Found {len(azure_paths)} azure paths. Sample: {azure_paths[:5]}")

container_client = blob_service_client.get_container_client(container)

def resolve_blob_name(container_client, requested_name):
    """
    Try direct exists() check first; if not found, search container for a blob
    whose basename matches requested_name (case-insensitive) or that contains it.
    Returns the resolved blob name (full path inside container) or None.
    """
    # direct check
    blob_client = container_client.get_blob_client(requested_name)
    try:
        if blob_client.exists():
            return requested_name
    except Exception:
        # ignore errors from exists() and try searching
        pass

    requested_basename = os.path.basename(requested_name).lower()
    print(f"Blob '{requested_name}' not found directly. Searching container for matching names...")
    matches = []
    for blob in container_client.list_blobs():
        name = blob.name
        basename = os.path.basename(name).lower()
        if basename == requested_basename or name.lower().endswith('/' + requested_basename) or requested_basename in name.lower():
            matches.append(name)
    if not matches:
        return None
    if len(matches) > 1:
        print(f"Multiple matches found for '{requested_basename}'; choosing the first one: {matches[:5]}")
    return matches[0]

download_paths = []

for path in azure_paths:
    print(f"Downloading {path}...")
    resolved = resolve_blob_name(container_client, path)
    if not resolved:
        print(f"ERROR: Blob for '{path}' not found in container '{container}'. Skipping.")
        continue

    blob_client = container_client.get_blob_client(resolved)
    # Save using the basename to avoid creating nested folders unless blob path explicitly required
    download_file_path = os.path.join('Downloads', os.path.basename(resolved))
    os.makedirs(os.path.dirname(download_file_path), exist_ok=True)
    # Download the blob
    try:
        with open(download_file_path, "wb") as download_file:
            download_stream = blob_client.download_blob()
            download_file.write(download_stream.readall())
        download_paths.append(download_file_path)
        print(f"Saved to {download_file_path}")
    except Exception as e:
        print(f"Failed to download blob '{resolved}': {e}")

Found 3656 azure paths. Sample: ['columpioscam3-2024-09-25_062223.mp4', 'columpioscam3-2024-09-25_062633.mp4', 'columpioscam2-2024-09-25_065853.mp4', 'columpioscam3-2024-09-25_070126.mp4', 'columpioscam2-2024-09-25_070533.mp4']
Blob 'columpioscam3-2024-09-25_062223.mp4' not found directly. Searching container for matching names...
ERROR: Blob for 'columpioscam3-2024-09-25_062223.mp4' not found in container 'crowdcounting'. Skipping.
Blob 'columpioscam3-2024-09-25_062633.mp4' not found directly. Searching container for matching names...
ERROR: Blob for 'columpioscam3-2024-09-25_062633.mp4' not found in container 'crowdcounting'. Skipping.
Blob 'columpioscam2-2024-09-25_065853.mp4' not found directly. Searching container for matching names...
ERROR: Blob for 'columpioscam2-2024-09-25_065853.mp4' not found in container 'crowdcounting'. Skipping.
Blob 'columpioscam3-2024-09-25_070126.mp4' not found directly. Searching container for matching names...


KeyboardInterrupt: 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import io
import re
from datetime import datetime
from typing import List, Dict, Optional
from azure.storage.blob import ContainerClient
from tqdm import tqdm

# =========================
# CONFIG — EDIT THESE
# =========================
ACCOUNT = "cienciaciudades2024"
CONTAINER = "crowdcounting"
SAS_TOKEN = (
    "sp=racwdl&st=2025-02-05T16:48:09Z&se=2026-04-02T00:48:09Z&spr=https&sv=2022-11-02&sr=c&"
    "sig=pXOsajenPg9lvILk6OozoyE%2BUz%2FDXivaBWwIioLyRVo%3D"
)
# Optional local CSV (if you don't have an in-memory df). Set CSV_PATH env var to override.
CSV_DEFAULT = "paths.csv"

# Download options
DOWNLOAD_DIR = "Downloads"     # target directory
PRESERVE_PREFIX = False        # True => keep blob virtual folders, False => flat files by basename
CHUNK_SIZE = 4 * 1024 * 1024   # 4 MiB per chunk


# =========================
# HELPERS
# =========================
def clean_name(s: str) -> str:
    """
    Normalize a candidate name (from df) to just the basename, stripped & de-quoted.
    """
    if s is None:
        return ""
    s = str(s)
    # Remove surrounding quotes and whitespace/newlines
    s = s.strip().strip('"').strip("'").strip()
    # Some rows might embed paths; use only the basename to match against blob index
    s = os.path.basename(s)
    # Collapse weird unicode spaces
    s = re.sub(r"\s+", " ", s).strip()
    return s


def build_container_client(account: str, container: str, sas_token: str) -> ContainerClient:
    container_url = f"https://{account}.blob.core.windows.net/{container}?{sas_token}"
    return ContainerClient.from_container_url(container_url)


def sanity_list(container_client: ContainerClient, n: int = 10) -> int:
    print("Sampling first blobs in container...")
    count = 0
    try:
        for blob in container_client.list_blobs():
            print("  -", blob.name)
            count += 1
            if count >= n:
                break
    except Exception as e:
        print(f"[FATAL] Could not list blobs: {e}")
        sys.exit(1)
    print(f"[OK] Listed {count} blob(s) as a sanity check.")
    return count


def build_basename_index(container_client: ContainerClient) -> Dict[str, List[str]]:
    """
    Build a mapping: lowercase(basename) -> [full blob paths].
    One pass over the container; streams results (does not load blob bodies).
    """
    print("Indexing container by basename (one pass)...")
    index: Dict[str, List[str]] = {}
    total = 0
    for blob in container_client.list_blobs():
        total += 1
        base = os.path.basename(blob.name).lower()
        index.setdefault(base, []).append(blob.name)
        if total % 2000 == 0:
            print(f"  Indexed {total} blobs...")
    print(f"[OK] Indexed {total} blob(s). Unique basenames: {len(index)}")
    return index


def resolve_blob_name(
    basename_index: Dict[str, List[str]],
    requested: str
) -> Optional[str]:
    """
    Resolve a requested (cleaned) filename by basename lookup (case-insensitive).
    If multiple matches exist, returns the first one (and warns).
    """
    key = requested.lower()
    matches = basename_index.get(key, [])
    if not matches:
        return None
    if len(matches) > 1:
        print(f"[WARN] Multiple matches for '{requested}'; taking the first: {matches[:5]}")
    return matches[0]


def ensure_dir(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)


def download_blob(
    container_client: ContainerClient,
    blob_path: str,
    dest_root: str,
    preserve_prefix: bool = False,
) -> str:
    """
    Download a single blob with progress bar. Compatible with older/newer azure-storage-blob:
    - use download_blob(max_concurrency=4) for speed
    - iterate stream.chunks() without passing chunk_size (some versions don't support it)
    """
    if preserve_prefix:
        rel = blob_path.replace("/", os.sep)
    else:
        rel = os.path.basename(blob_path)

    local_path = os.path.join(dest_root, rel)
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    blob_client = container_client.get_blob_client(blob_path)

    props = blob_client.get_blob_properties()
    size = props.size or 0

    # parallelize server-side downloads a bit; safe across versions
    stream = blob_client.download_blob(max_concurrency=4)

    with open(local_path, "wb") as out, tqdm(
        total=size, unit="B", unit_scale=True, desc=os.path.basename(local_path)
    ) as pbar:
        # IMPORTANT: no chunk_size arg here for broad compatibility
        for chunk in stream.chunks():
            out.write(chunk)
            pbar.update(len(chunk))

    return local_path


def gather_azure_paths_from_df() -> Optional[List[str]]:
    """
    If a global df is present (pandas DataFrame), use it to extract target names.
    Looks for 'video_path' first; falls back to 'id' (splitting on ':').
    """
    g = globals()
    if "df" not in g:
        return None
    try:
        import pandas as pd  # noqa: F401
    except Exception:
        return None

    _df = g["df"]
    if not hasattr(_df, "columns"):
        return None

    if "video_path" in _df.columns and _df["video_path"].notna().any():
        series = _df["video_path"].dropna().astype(str)
        values = [clean_name(v) for v in series.tolist()]
    else:
        # fallback to 'id'
        col = "id" if "id" in _df.columns else None
        if not col:
            return None
        series = _df[col].dropna().astype(str)
        values = []
        for s in series.tolist():
            left = s.split(":")[0] if ":" in s else s
            values.append(clean_name(left))

    seen = set()
    out = []
    for v in values:
        if v and v not in seen:
            seen.add(v)
            out.append(v)
    return out


def gather_azure_paths_from_csv(csv_path: str) -> Optional[List[str]]:
    """
    Load from CSV if present, looking for 'video_path' or 'id'.
    """
    if not os.path.exists(csv_path):
        return None
    try:
        import pandas as pd
        df_csv = pd.read_csv(csv_path)
    except Exception as e:
        print(f"[WARN] Could not read CSV '{csv_path}': {e}")
        return None

    if "video_path" in df_csv.columns and df_csv["video_path"].notna().any():
        series = df_csv["video_path"].dropna().astype(str)
        values = [clean_name(v) for v in series.tolist()]
    elif "id" in df_csv.columns and df_csv["id"].notna().any():
        series = df_csv["id"].dropna().astype(str)
        values = []
        for s in series.tolist():
            left = s.split(":")[0] if ":" in s else s
            values.append(clean_name(left))
    else:
        print(f"[WARN] CSV '{csv_path}' has neither 'video_path' nor 'id' with values.")
        return None

    seen = set()
    out = []
    for v in values:
        if v and v not in seen:
            seen.add(v)
            out.append(v)
    return out


# =========================
# MAIN
# =========================
def main():
    # 1) Connect (container-scoped)
    print("[INFO] Connecting to container…")
    container_client = build_container_client(ACCOUNT, CONTAINER, SAS_TOKEN)

    # 2) Sanity listing (also surfaces permission issues early)
    sanity_list(container_client, n=5)

    # 3) Build basename index (one pass)
    basename_index = build_basename_index(container_client)

    # 4) Build desired target list (from df or CSV)
    azure_paths = None

    # Prefer in-memory df if present
    azure_paths = gather_azure_paths_from_df()
    if azure_paths:
        print(f"[OK] Collected {len(azure_paths)} target name(s) from in-memory df.")
    else:
        # Fall back to CSV
        csv_path = os.environ.get("CSV_PATH", CSV_DEFAULT)
        azure_paths = gather_azure_paths_from_csv(csv_path)
        if azure_paths:
            print(f"[OK] Collected {len(azure_paths)} target name(s) from CSV: {csv_path}")

    if not azure_paths:
        print(
            "[FATAL] No targets to download. Provide a pandas DataFrame named 'df' "
            "with a 'video_path' or 'id' column, or create a CSV (default 'paths.csv')."
        )
        sys.exit(1)

    print(f"[INFO] Sample targets: {azure_paths[:5]}")

    # 5) Resolve & download
    os.makedirs(DOWNLOAD_DIR, exist_ok=True)
    downloaded = 0
    missing = 0

    for target in azure_paths:
        # Try exact basename resolution via index
        resolved = resolve_blob_name(basename_index, target)
        if not resolved:
            print(f"[MISS] Not found by basename: '{target}'")
            missing += 1
            continue

        try:
            local_path = download_blob(
                container_client,
                resolved,
                dest_root=DOWNLOAD_DIR,
                preserve_prefix=PRESERVE_PREFIX,
                chunk_size=CHUNK_SIZE
            )
            print(f"[OK] Saved: {local_path}")
            downloaded += 1
        except Exception as e:
            print(f"[ERR] Failed to download '{resolved}': {e}")

    print("\n========== SUMMARY ==========")
    print(f"Requested:  {len(azure_paths)}")
    print(f"Downloaded: {downloaded}")
    print(f"Missing:    {missing}")
    print(f"Output dir: {os.path.abspath(DOWNLOAD_DIR)}")
    print("=============================")


if __name__ == "__main__":
    main()

[INFO] Connecting to container…
Sampling first blobs in container...
  - banorte_cam1-2024-10-24 16:49:52.mp4
  - banorte_cam1-2024-10-24 16:52:25.mp4
  - banorte_cam1-2024-10-24 16:52:52.mp4
  - banorte_cam1-2024-10-24 16:54:37.mp4
  - banorte_cam1-2024-10-24 16:56:22.mp4
[OK] Listed 5 blob(s) as a sanity check.
Indexing container by basename (one pass)...
  Indexed 2000 blobs...
  Indexed 4000 blobs...
  Indexed 6000 blobs...
  Indexed 8000 blobs...
  Indexed 10000 blobs...
  Indexed 12000 blobs...
  Indexed 14000 blobs...
  Indexed 16000 blobs...
  Indexed 18000 blobs...
  Indexed 20000 blobs...
  Indexed 22000 blobs...
  Indexed 24000 blobs...
  Indexed 26000 blobs...
  Indexed 28000 blobs...
  Indexed 30000 blobs...
  Indexed 32000 blobs...
  Indexed 34000 blobs...
  Indexed 36000 blobs...
  Indexed 38000 blobs...
  Indexed 40000 blobs...
  Indexed 42000 blobs...
  Indexed 44000 blobs...
  Indexed 46000 blobs...
  Indexed 48000 blobs...
  Indexed 50000 blobs...
  Indexed 52000 blo

columpios_cam4-2024-12-09_07-14-00.mp4:   0%|          | 0.00/1.80M [00:00<?, ?B/s]


[ERR] Failed to download 'columpios_cam4-2024-12-09_07-14-00.mp4': StorageStreamDownloader.chunks() got an unexpected keyword argument 'chunk_size'


columpioscam3-2024-12-09_07-15-00.mp4:   0%|          | 0.00/1.95M [00:00<?, ?B/s]


[ERR] Failed to download 'columpioscam3-2024-12-09_07-15-00.mp4': StorageStreamDownloader.chunks() got an unexpected keyword argument 'chunk_size'


...

columpioscam3-2024-12-14_11-58-00.mp4:   0%|          | 0.00/2.76M [00:00<?, ?B/s]


[ERR] Failed to download 'columpioscam3-2024-12-14_11-58-00.mp4': StorageStreamDownloader.chunks() got an unexpected keyword argument 'chunk_size'


columpioscam3-2024-12-14_12-01-00.mp4:   0%|          | 0.00/2.70M [00:00<?, ?B/s]


[ERR] Failed to download 'columpioscam3-2024-12-14_12-01-00.mp4': StorageStreamDownloader.chunks() got an unexpected keyword argument 'chunk_size'


KeyboardInterrupt: 